In [4]:
%pwd

'c:\\H\\tinychat-lora'

In [ ]:
!pip install -r requirements.txt

In [6]:
import torch
print("当前是否有GPU可用：", torch.cuda.is_available())
print("GPU名称：", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "无")


当前是否有GPU可用： False
GPU名称： 无


### 数据转换，把"BelleGroup/train_0.5M_CN"格式换成跟QWen一样

In [ ]:
import json
from datasets import load_dataset
from tqdm import tqdm


output_file="data/converted_belle.jsonl"
dataset = load_dataset("BelleGroup/train_0.5M_CN", trust_remote_code=True, split="train")

In [ ]:
with open(output_file, "w", encoding="utf-8") as fout:
    for example in tqdm(dataset):
        instruction = example.get("instruction", "").strip()
        output = example.get("output", "").strip()

        if not instruction or not output:
            continue

        messages = [
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": output}
        ]
        fout.write(json.dumps({"messages": messages}, ensure_ascii=False) + "\n")

100%|██████████| 519255/519255 [00:41<00:00, 12544.42it/s]


In [ ]:
!head -n 10 data/converted_belle.jsonl

{"messages": [{"role": "user", "content": "给定一个英文句子，翻译成中文。\nI love to learn new things every day."}, {"role": "assistant", "content": "我每天喜欢学习新事物。"}]}
{"messages": [{"role": "user", "content": "给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”"}, {"role": "assistant", "content": "“明天的会议在10点开始，记得准时到达。”"}]}
{"messages": [{"role": "user", "content": "根据以下信息创建一个新的电子邮件账户：您的用户名应该包含您的姓氏和您的生日，在.com域中注册一个电子邮件地址，并将生成的用户名和密码作为输出提供。\n姓氏：李\n生日：1990年1月1日"}, {"role": "assistant", "content": "生成的用户名应该是li19900101，并在.com域中注册电子邮件地址。由于安全和隐私原因，我无法提供您所需的密码。请确保您选择一个强密码并将其安全地记录下来。"}]}
{"messages": [{"role": "user", "content": "根据给定的文字生成一篇关于人工智能的文章，包括定义、历史和应用。\n人工智能是指使计算机具有智慧的能力。它的历史可以追溯到20世纪50年代。今天，人工智能在很多领域包括医学、金融和机器人等方面得到了广泛的应用。"}, {"role": "assistant", "content": "人工智能是指让计算机具备像人类一样思考、判断和学习的能力。这种技术的历史可以追溯到20世纪50年代，当时计算机科学家开始试图模拟人类智能的思维过程。\n在过去的几十年中，人工智能技术已经得到了广泛的应用。在医学领域，人工智能被用来诊断疾病和制定个性化的治疗计划。在金融行业，人工智能被用来分析市场趋势和进行股票交易。在工业和制造业中，人工智能被用来管理生产线和控制机器人。\n随着技术的不断进步，人工智能的应用范围也在不断扩大。它正在被越来越多的行业和领域所采用，这将会为人们的生活带来巨大的改变。"}]}

### LoRA微调

##### 1. 选择模型并下载：Qwen1.5-0.5B-Chat
体积小（1.9GB），支持中文，结构和 Qwen 系列保持一致，非常适合 LoRA 微调。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen1.5-0.5B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True, torch_dtype="auto")

##### 2. 应用 LoRA

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,                             # 降维后维度（rank）
    lora_alpha=16,                  # 缩放因子（实际权重乘以 alpha/r）
    target_modules=["c_attn", "q_proj", "v_proj"],  # 指定插入 LoRA 的层名
    lora_dropout=0.05,              # 训练时的 dropout
    bias="none",                    # 不训练 bias
    task_type=TaskType.CAUSAL_LM    # 指定任务类型是语言建模
)
model = get_peft_model(model, lora_config)


##### 3. 使用 transformers 的 Trainer API 来训练加了 LoRA 的模型，先加载数据集

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/converted_belle.jsonl", split='train')

##### 4. 构建模板：将 instruction + output 合并为 prompt 格式

In [ ]:
def format_example(example):
    messages = example["messages"]
    user_msg = ""
    assistant_msg = ""
    for i in range(len(messages) - 1):
        if messages[i]["role"] == "user" and messages[i+1]["role"] == "assistant":
            user_msg = messages[i]["content"]
            assistant_msg = messages[i+1]["content"]
            break
    return {
        "text": f"### 指令：\n{user_msg}\n\n### 回答：\n{assistant_msg}"
    }


dataset = dataset.map(format_example)

Map:   0%|          | 0/519229 [00:00<?, ? examples/s]

In [ ]:
for i in range(3):
    print(dataset[i]["text"])
    print("=" * 50)


### 指令：
给定一个英文句子，翻译成中文。
I love to learn new things every day.

### 回答：
我每天喜欢学习新事物。
### 指令：
给定一个文字输入，将其中的所有数字加1。
“明天的会议在9点开始，记得准时到达。”

### 回答：
“明天的会议在10点开始，记得准时到达。”
### 指令：
根据以下信息创建一个新的电子邮件账户：您的用户名应该包含您的姓氏和您的生日，在.com域中注册一个电子邮件地址，并将生成的用户名和密码作为输出提供。
姓氏：李
生日：1990年1月1日

### 回答：
生成的用户名应该是li19900101，并在.com域中注册电子邮件地址。由于安全和隐私原因，我无法提供您所需的密码。请确保您选择一个强密码并将其安全地记录下来。


##### 5. 对belle数据集分词

In [ ]:
def tokenize(example):
    result = tokenizer(
        example["text"],
        max_length=512,
        truncation=True,
        padding="max_length",  # 或 "longest"，但训练时建议统一长度
    )
    result["labels"] = result["input_ids"].copy()  # 这是训练时必须要有的，标签就是语言模型的输入
    return result


tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/519229 [00:00<?, ? examples/s]

##### 6. 使用 Trainer 开始训练， LoRA微调

Qwen1.5-0.5B-Chat（约5亿参数）；

数据集大小：Belle 0.5M 中文约50万条样本；

In [ ]:
from transformers import TrainingArguments, Trainer
import torch
"""
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # 等效于 batch_size=16
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    learning_rate=2e-4,
    fp16=True,
    report_to="none",  # 不推送到 wandb 等平台
    remove_unused_columns=False,
)"""

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4, # 增多的话能减少accumulation步数，单步计算更高效
    gradient_accumulation_steps=4,  # 等效于 batch_size=16
    max_steps=3000,
    save_strategy="epoch",                 # 每个 epoch 保存一次
    logging_steps=50,
    logging_dir="./logs",
    save_steps=500,
    save_total_limit=3,
    learning_rate=2e-4,
    report_to="none",                      # 不推送到 wandb 等平台
    fp16=True,       # 使用混合精度
)

# 只训练前 1000 条样本（调试）
# small_dataset = tokenized_dataset.select(range(1000))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)
trainer.train()
# trainer.train(resume_from_checkpoint=True)




<ipython-input-35-e5e853b5f53b>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,0.409900
100,0.414000
150,0.448600
200,0.426500
250,0.427600
300,0.420100
350,0.405300
400,0.423200
450,0.397500
500,0.417000


TrainOutput(global_step=3000, training_loss=0.41133735911051433, metrics={'train_runtime': 5322.0911, 'train_samples_per_second': 9.019, 'train_steps_per_second': 0.564, 'total_flos': 4.559216836608e+16, 'train_loss': 0.41133735911051433, 'epoch': 0.09244422531739184})

In [ ]:
!nvidia-smi

Sun May 18 00:59:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   71C    P0             30W /   70W |   13190MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 从 Checkpoint 恢复验证

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer
from peft import PeftModel

# 加载 base model
base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", trust_remote_code=True)

# 加载最后的 checkpoint（假设是 3000）
checkpoint_path = "./results/checkpoint-3000"
model = PeftModel.from_pretrained(base_model, checkpoint_path)
model.eval()

from transformers import TextStreamer

streamer = TextStreamer(tokenizer)

prompt = "### 指令：\n把这句话翻译成中文：I love learning new things every day.\n\n### 回答："
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    streamer=streamer
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

### 指令：
把这句话翻译成中文：I love learning new things every day.

### 回答：我每天都喜欢学习新事物。<|endoftext|>


In [11]:
prompt = "### 指令：\n我是谁\n\n### 回答："
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    streamer=streamer
)

### 指令：
我是谁

### 回答：您是您的名字。<|endoftext|>


In [14]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", device_map="auto")
base_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")

lora_model = PeftModel.from_pretrained(base_model, "results/checkpoint-3000")
tokenizer = base_tokenizer  # tokenizer 是一样的

def compare_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 原始模型
    base_out = base_model.generate(**inputs, max_new_tokens=100)
    base_text = tokenizer.decode(base_out[0], skip_special_tokens=True)

    # LoRA 微调模型
    lora_out = lora_model.generate(**inputs, max_new_tokens=100)
    lora_text = tokenizer.decode(lora_out[0], skip_special_tokens=True)

    print("\nPrompt:", prompt)
    print("=== 原始模型输出 ===")
    print(base_text)
    print("=== LoRA 微调输出 ===")
    print(lora_text)

prompts = [
        "### 指令：\n将下面这句话翻译成中文：\nLearning a new language can be fun and rewarding.### 回答：",
        "请将下面这句话翻译成中文：\nThe quick brown fox jumps over the lazy dog.",
        "翻成中文：She sells seashells by the seashore.",
        "### 指令：\n翻译：When life gives you lemons, make lemonade.\n\n### 回答：\n",
        "### 指令：\n请解释这句话的含义：\n“Knowledge is power.”\n### 回答：\n"
      ]
for p in prompts:
  compare_response(p)



Prompt: ### 指令：
将下面这句话翻译成中文：
Learning a new language can be fun and rewarding.### 回答：
=== 原始模型输出 ===
### 指令：
将下面这句话翻译成中文：
Learning a new language can be fun and rewarding.### 回答：学习新语言可以有趣又令人欣慰。
=== LoRA 微调输出 ===
### 指令：
将下面这句话翻译成中文：
Learning a new language can be fun and rewarding.### 回答：学习一门新的语言可以是有趣和有回报的。

Prompt: 请将下面这句话翻译成中文：
The quick brown fox jumps over the lazy dog.
=== 原始模型输出 ===
请将下面这句话翻译成中文：
The quick brown fox jumps over the lazy dog.
=== LoRA 微调输出 ===
请将下面这句话翻译成中文：
The quick brown fox jumps over the lazy dog.

Prompt: 翻成中文：She sells seashells by the seashore.
=== 原始模型输出 ===
翻成中文：She sells seashells by the seashore.
=== LoRA 微调输出 ===
翻成中文：She sells seashells by the seashore.

Prompt: ### 指令：
翻译：When life gives you lemons, make lemonade.

### 回答：

=== 原始模型输出 ===
### 指令：
翻译：When life gives you lemons, make lemonade.

### 回答：
"Life hands you lemons, make lemonade."
=== LoRA 微调输出 ===
### 指令：
翻译：When life gives you lemons, make lemonade.

### 回答：
“当生活给你柠檬，就酿制柠檬汁。”

Prompt: ### 

# 轻量版 RAG 推理流程

##### 1. 加载医疗问答库

In [6]:
import json

corpus = []
with open("data/medical.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        for convo in item["conversations"]:
            if convo["role"] == "user":
                question = convo["content"]
            elif convo["role"] == "assistant":
                answer = convo["content"]
                corpus.append({"question": question, "answer": answer})


##### 2. 建立简单的向量索引

In [ ]:
!pip install -U sentence-transformers

In [10]:
from sentence_transformers import SentenceTransformer
import numpy as np

questions = [item["question"] for item in corpus]
# 加载更轻量的模型
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" , trust_remote_code=True)  # CPU 可跑的小模型200M，多语言
# 使用批处理 + 转为 numpy，防止 CPU 内存爆炸
corpus_embeddings = embedder.encode(
    questions,
    batch_size=32,                # 可根据机器内存调大调小
    convert_to_numpy=True,       # 避免 GPU 或 PyTorch tensor 堆积
    show_progress_bar=True
)

# 可选：保存向量（如果你要做RAG或之后用）
np.save("corpus_embeddings.npy", corpus_embeddings)


Batches: 100%|██████████| 790/790 [03:29<00:00,  3.78it/s]


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def search_similar(query, top_k=5):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    similarities = cosine_similarity(query_embedding, corpus_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]

    for idx in top_indices:
        print(f"[score: {similarities[idx]:.4f}] Q: {corpus[idx]['question']}\nA: {corpus[idx]['answer']}\n")

search_similar("孕期如何控制血糖？")

[score: 0.8175] Q: 孕期血糖一直在7.0----8.0之间对胎儿有影响吗?
A: 孕期血糖控制不良可能会对胎儿产生一些影响。孕期血糖一直在7.0-8.0mmol/L之间，这个水平相对较高，可能会增加以下风险：

1. 大儿症：高血糖会导致胎儿体重过大，这可能会增加分娩困难和剖腹产的风险。

2. 早产：高血糖可能会导致早产，早产儿可能会面临呼吸困难等问题。

3. 低血糖：出生后，婴儿可能会出现低血糖，需要特殊的护理。

4. 黄疸：新生儿可能会出现黄疸，虽然这是一个常见的新生儿问题，但是如果严重可能需要治疗。

5. 儿童和成年后的肥胖和糖尿病：如果母亲在孕期血糖控制不良，孩子在儿童期和成年后患肥胖和糖尿病的风险会增加。

因此，建议您尽快与医生联系，制定合适的血糖管理计划，包括饮食、运动和可能的药物治疗。同时，定期进行产检，监测胎儿的发育情况，以确保母婴健康。

[score: 0.7857] Q: 妊娠糖尿病对胎儿有什么影响?
A: 妊娠糖尿病是指在怀孕期间出现或者诊断的糖尿病，它可能会对母亲和胎儿产生一些影响。以下是一些可能的影响：

1. 宏观胎儿：由于母体血糖高，胎儿会产生更多的胰岛素以应对高血糖，这可能导致胎儿体重过大，称为宏观胎儿。这可能会增加分娩的困难和剖腹产的可能性。

2. 早产：妊娠糖尿病可能会增加早产的风险。早产儿可能会面临呼吸困难、黄疸、低血糖等问题。

3. 呼吸窘迫综合症：宏观胎儿可能会有呼吸窘迫综合症的风险，这是因为他们的肺部可能没有完全发育。

4. 低血糖（新生儿低血糖）：由于胎儿在母体内已经习惯了高血糖环境，出生后可能会出现低血糖，需要立即治疗。

5. 肥胖和二型糖尿病：长期来看，这些孩子在成年后可能更容易发胖，且更有可能患上二型糖尿病。

6. 死亡风险：虽然罕见，但妊娠糖尿病可能会增加胎儿死亡的风险。

为了降低这些风险，孕妇需要定期监测血糖，遵循医生的饮食和运动建议，有时可能需要使用胰岛素。在分娩后，还需要继续监测血糖，并进行健康的生活方式调整，以降低未来发展为2型糖尿病的风险。

[score: 0.7766] Q: 妊娠合并糖尿病对胎儿都有哪些影响？
A: 妊娠合并糖尿病，也被称为妊娠糖尿病，对胎儿可能有以下影响：

1. 宏观胎儿：由于母体血糖高，胎儿胰岛素分泌增多，导致胎儿生长过快，体重增加，

##### 3. 构造 RAG Prompt 并调用 LoRA 模型生成回答

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# 原始 Qwen 模型路径
base_model_path = "Qwen/Qwen1.5-0.5B-Chat"

# LoRA 微调后的 checkpoint 路径
lora_model_path = "./results/checkpoint-3000"

# 加载 base 模型
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

# 加载 LoRA 并合并
model = PeftModel.from_pretrained(base_model, lora_model_path)
model = model.merge_and_unload()  # 合并 LoRA 权重
model.eval()

# 保存合并后的模型
save_path = "./tinychat_lora"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


c:\Users\wysxx\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\wysxx\.cache\huggingface\hub\models--Qwen--Qwen1.5-0.5B-Chat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may 

('./tinychat_lora\\tokenizer_config.json',
 './tinychat_lora\\special_tokens_map.json',
 './tinychat_lora\\vocab.json',
 './tinychat_lora\\merges.txt',
 './tinychat_lora\\added_tokens.json',
 './tinychat_lora\\tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "./tinychat_lora",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat", trust_remote_code=True)

def retrieve_context(query, top_k=3):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = cosine_similarity(query_embedding.cpu().numpy().reshape(1, -1), corpus_embeddings)[0]
    top_indices = np.argsort(cos_scores)[::-1][:top_k]
    return [corpus[idx] for idx in top_indices]

c:\Users\wysxx\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
def rag_ask(query):
    context = retrieve_context(query)
    context_text = "\n\n".join([f"问：{item['question']}\n答：{item['answer']}" for item in context])

    prompt = f"""你是一个医疗问答助手，请根据以下已知问答来回答用户的新问题。
### 已知问答：
{context_text}
### 用户问题：
{query}
### 回答："""

    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    streamer = TextStreamer(tokenizer)

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        do_sample=True,          # 采样模式，配合top_p使用
        top_p=0.8,
        temperature=0.7,
        streamer=streamer
    )
    # 只解码生成的部分，跳过 prompt 长度
    generated_text = tokenizer.decode(output[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print(generated_text)


rag_ask("特异性皮炎能根治吗？")  # chat gpt 生成，有点问题


你是一个医疗问答助手，请根据以下已知问答来回答用户的新问题。
### 已知问答：
问：脑包虫病能完全治愈吗
答：脑包虫病，也称为脑囊虫病或神经囊虫病，是由于人体摄入了包虫病的幼虫，这些幼虫通过血液循环进入大脑，形成包虫囊肿而引发的一种疾病。这是一种严重的寄生虫病，如果不及时治疗，可能会对患者的生命造成威胁。

脑包虫病的治疗主要包括药物治疗和手术治疗。药物治疗主要是使用抗寄生虫药物，如阿苯达唑（Albendazole）和咪唑类药物，这些药物可以杀死包虫幼虫，减小囊肿的大小。手术治疗则是直接切除大脑中的包虫囊肿，但由于大脑是人体的重要器官，手术风险较大。

至于脑包虫病能否完全治愈，这取决于多种因素，包括病情的严重程度、囊肿的位置和大小、治疗的及时性等。在一些情况下，通过药物治疗和手术治疗，可以有效地控制病情，甚至实现完全治愈。然而，在一些复杂或严重的情况下，可能无法完全治愈，需要长期的治疗和管理。

总的来说，脑包虫病的治疗需要医生根据患者的具体情况制定个性化的治疗方案。如果你或你的亲友被诊断为脑包虫病，应尽快寻求专业医疗帮助，以获取最佳的治疗方案。

问：请问股癣会不会自愈吗？
答：股癣，也被称为脚癣或环状肌炎，是一种由真菌引起的皮肤感染。这种真菌在温暖潮湿的环境中生长，如公共淋浴和更衣室。

股癣可能不会自行消失，通常需要治疗。如果不进行治疗，症状可能会持续，甚至可能会恶化。此外，未经治疗的股癣可能会传播给他人。

治疗股癣的方法包括：

1. 局部抗真菌药膏、霜剂或喷雾：这些可以在大多数药店购买，不需要处方。例如克霉唑、咪康唑等。

2. 口服抗真菌药物：如果症状严重或持续不愈，医生可能会开具口服抗真菌药物的处方。

3. 避免与他人共享毛巾、衣物或其他个人物品，以防止感染的传播。

4. 保持皮肤干燥清洁，特别是在运动或出汗后。

如果你怀疑自己有股癣，建议你尽快就医。医生可以通过检查皮肤样本来确诊，并建议适当的治疗方法。

问：你好癫痫病能够不治而愈吗？
答：癫痫是一种慢性疾病，由于大脑神经元的异常放电引起。这种疾病的特点是反复发作的癫痫发作，这可能包括肌肉抽搐、感觉改变、行为改变或意识丧失。

癫痫的治疗目标是控制症状，减少发作频率和严重程度，以及改善生活质量。大多数癫痫患者可以通过药物治疗来控制他们的症状。这些药物，被称为抗癫痫药，包括卡马西平、苯妥英

In [ ]:
def rag_ask_ds(query):
    context = retrieve_context(query)
    context_text = "\n".join([f"问：{item['question']}\n答：{item['answer']}" for item in context])

    prompt = [
        {"role": "system", "content": "你是一个医疗问答助手，请根据已知信息回答问题"},
        {"role": "user", "content": f"已知信息：\n{context_text}\n\n问题：{query}"}
    ]
    
    inputs = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    streamer = TextStreamer(tokenizer)
    
    output = model.generate(
        inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        temperature=0.3,
        streamer=streamer,
    )
    
    # 提取生成部分（跳过 prompt）
    generated = output[0][inputs.shape[-1]:]
    print(tokenizer.decode(generated, skip_special_tokens=True))

rag_ask_ds("特异性皮炎能根治吗？")  # 这个是可以的

IndexError: too many indices for tensor of dimension 2